In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
import re
import pickle
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import json
from scipy.stats import linregress
from sklearn.cluster import KMeans

In [2]:
dates_times_path = 'dates_times.pickle'
density_lines_path = 'trend_lines.pickle'
json_path = 'json_geo.json'
animation_path = 'animation.pickle'

In [3]:
with open(density_lines_path, 'rb') as handle:
    b = pickle.load(handle)
df_dict = b
with open(dates_times_path, 'rb') as handle:
    b = pickle.load(handle)
dates_times = b
with open(animation_path, 'rb') as handle:
    b = pickle.load(handle)
df_geo = b
with open(json_path) as data_file:
    new_jfile = json.load(data_file)
    
dict_keys = []
dict_keys = df_dict.keys()

In [4]:
trends = {}
dates = []
for i in range(0,len(dates_times),3):
    date = dates_times[i][0]
    dates.append(date)
    vals = []
    for key in df_dict.keys():
        df = df_dict[key]
        vals.extend((df.loc[df['dates'] == date].values[0][1:4].astype(float)))
    #print(vals)
    q_25 = np.percentile(vals, 25)
    q_50 = np.percentile(vals, 50)
    q_75 = np.percentile(vals, 75)
    trends[date] = (q_25, q_50, q_75)
print(trends)

{'2020-11-15': (14.377518034233445, 19.34190077125552, 26.12258727994858), '2020-11-16': (15.51352938360166, 20.225075286969158, 28.754879736808917), '2020-11-17': (16.358617693120486, 21.04957751888935, 29.410177730253526), '2020-11-18': (15.924658902189915, 20.773337771800534, 29.13530454073888), '2020-11-19': (15.493945303564836, 21.37739610108188, 29.423495402298435), '2020-11-20': (16.05007403268148, 20.475928879214194, 29.09204503379467), '2020-11-21': (14.426585714544307, 18.92150840021982, 25.34353173422391), '2020-11-22': (13.774325163960274, 17.9506342332721, 24.381102619237694), '2020-11-23': (14.36305721940268, 19.00000229562497, 27.333711173873454), '2020-11-24': (14.635228651340082, 19.313964218607687, 26.67201986013353), '2020-11-25': (14.501273076280187, 18.982580434760113, 26.346713615687385), '2020-11-26': (13.26605152471323, 16.655769440130378, 20.811110748422067), '2020-11-27': (12.891572468221767, 16.35462902020161, 22.818605022662446), '2020-11-28': (12.9969715621

In [16]:
trends_lower = []
trends_upper = []
trends_median = []
for i in range(0,len(dates_times),3):
    date = dates_times[i][0]
    lower_count = 0
    upper_count = 0
    median_count = 0
    for key in df_dict.keys():
        df = df_dict[key]
        vals = ((df.loc[df['dates'] == date].values[0][1:4].astype(float)))
        if (np.percentile(vals, 25)) <= trends[date][0]:
            lower_count = lower_count + 1
        elif ((np.percentile(vals, 75)) >= trends[date][2]):
            upper_count = upper_count + 1
        else:
            median_count = median_count + 1
    trends_lower.append(lower_count)
    trends_upper.append(upper_count)
    trends_median.append(median_count)

In [17]:

fig = go.Figure()
fig.add_trace(go.Scatter(y=trends_lower, x=dates,
                    mode='lines',
                    name='decrease'))
fig.add_trace(go.Scatter(y=trends_upper, x=dates,
                    mode='lines',
                    name='increase'))
fig.add_trace(go.Scatter(y=trends_median, x=dates,
                    mode='lines',
                    name='median'))

fig.show()